# Team Roster Analyzer using Sleepers api and RotoUnderworld lifetime values

## set up

In [95]:
import numpy as np
import pandas as pd

from sleeper_wrapper import League
import requests


In [96]:
league = League(692104824862367744)

In [97]:
data_in = './data_in/'
data_out = './data_out/'
data_postfix = 'dynasty-ppr-rankings'
league_id=692104824862367744

In [98]:
ALL = pd.read_csv('{}All-{}.csv'.format(data_in,data_postfix))

## add initial picks with 3rd round reverse

In [99]:
fwards=[0,1,2,3,4,5,6,7,8,9,10,11]
bwards=[11,10,9,8,7,6,5,4,3,2,1,0]

In [100]:
picks={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[]}

In [101]:
players={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[]}
players_val={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[]}

In [102]:
for x in range(1,5):
    for k,v in enumerate(fwards):
        picks[k+1].append(x)

In [103]:
picks

{1: [1, 2, 3, 4],
 2: [1, 2, 3, 4],
 3: [1, 2, 3, 4],
 4: [1, 2, 3, 4],
 5: [1, 2, 3, 4],
 6: [1, 2, 3, 4],
 7: [1, 2, 3, 4],
 8: [1, 2, 3, 4],
 9: [1, 2, 3, 4],
 10: [1, 2, 3, 4],
 11: [1, 2, 3, 4],
 12: [1, 2, 3, 4]}

### roster 2 slot

In [104]:
draft_meta=requests.get("https://api.sleeper.app/v1/draft/692104825852223488")

In [105]:
roster_id_slots=draft_meta.json()['slot_to_roster_id']
roster_id_slots = {v:int(k) for k, v in roster_id_slots.items()}
roster_id_slots

{12: 9, 7: 8, 10: 7, 2: 6, 9: 5, 5: 4, 4: 3, 8: 2, 1: 12, 6: 11, 3: 10, 11: 1}

In [106]:
def roster_2_slot(x):
    return roster_id_slots[x]

In [107]:
def slot_2_roster(x):
    slot_2_roster = {v:int(k) for k, v in roster_id_slots.items()}
    return slot_2_roster[x]

## Pull the traded picks so far from the api

In [108]:
traded=requests.get(f"https://api.sleeper.app/v1/league/{league_id}/traded_picks")

In [129]:
traded.json()

list

In [110]:
trade_table=pd.json_normalize(traded.json())
trade_table=trade_table.rename(columns={'round':'Rou'})

### assign the pick number to the traded pick

In [111]:
trade_table=trade_table[trade_table['season']!='2021']

In [112]:
trade_table

,season,Rou,roster_id,previous_owner_id,owner_id
26,2022,1,4,4,1
34,2022,1,5,6,3
44,2022,2,6,6,4
67,2022,1,8,8,7
68,2022,2,8,8,3
90,2022,1,12,12,7


### remove the pick from the old team, and give it to the new team

In [113]:
for row in trade_table.itertuples():
    picks[row.roster_id].remove(row.Rou)
    picks[row.owner_id].append(row.Rou)

In [115]:
picks

{1: [1, 2, 3, 4, 1],
 2: [1, 2, 3, 4],
 3: [1, 2, 3, 4, 1, 2],
 4: [2, 3, 4, 2],
 5: [2, 3, 4],
 6: [1, 3, 4],
 7: [1, 2, 3, 4, 1, 1],
 8: [3, 4],
 9: [1, 2, 3, 4],
 10: [1, 2, 3, 4],
 11: [1, 2, 3, 4],
 12: [2, 3, 4]}

In [211]:
pick_val={1:96,
2:51,
3:29,
4:16}

In [207]:
#dictionary to fix name discrepencies
name_dict={
        'DK Metcalf':"D.K. Metcalf",
        'DeVonta Smith':'Devonta Smith',
        'Laviska Shenault':'LaViska Shenault',
        'Kenneth Gainwell':'Kenny Gainwell',
        'La\'Mical Perine':'Lamical Perine'
}

## Pull the rosters from sleepers api

In [116]:
draft=requests.get(f"https://api.sleeper.app/v1/league/{league_id}/rosters")

In [2]:
#draft.json()

In [118]:
draft_df=pd.json_normalize(draft.json())

In [119]:
draft_df

,taxi,starters,roster_id,reserve,players,player_map,owner_id,league_id,co_owners,settings.wins,...,settings.ties,settings.losses,settings.fpts,metadata.restrict_pn_scoring_starters_only,metadata.allow_pn_scoring,metadata,metadata.p_nick_4034,metadata.p_nick_3321,metadata.p_nick_3199,metadata.p_nick_1426
0,"[7562, 7568, 7594, 7670]","[7523, 7588, 3664, 6786, 6783, 6886, 7553, 756...",1,None,"[2306, 2460, 3286, 3664, 4157, 4622, 4943, 678...",None,565684665600733184,692104824862367744,None,0,...,0,0,0,on,off,NaN,NaN,NaN,NaN,NaN
1,"[7579, 7608, 7703, 7956]","[4046, 3164, 5892, 2216, 5848, 5045, 2505, 182...",2,None,"[1706, 1825, 2028, 2118, 2216, 2319, 2325, 237...",None,464605759842283520,692104824862367744,None,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,"[3294, 6151, 3969, 2309, 4950, 6805, 5857, 693...",3,None,"[2309, 3294, 3969, 4435, 4571, 4718, 4950, 500...",None,315559272291237888,692104824862367744,None,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[7556, 7609, 7720, 7944]","[5849, 6845, 7543, 5846, 5872, 6789, 5844, 679...",4,None,"[1833, 2152, 2583, 2711, 3271, 5185, 5844, 584...",None,694764145630867456,692104824862367744,None,0,...,0,0,0,on,on,NaN,NaN,NaN,NaN,NaN
4,"[7536, 7558, 7560, 7605]","[4881, 5850, 6955, 2133, 6801, 6819, 4602, 223...",5,None,"[2133, 2238, 2257, 2588, 4602, 4881, 4973, 525...",None,694780713802743808,692104824862367744,None,0,...,0,0,0,on,on,NaN,NaN,NaN,NaN,NaN
5,"[7569, 7574, 7595, 7604]","[4017, 4018, 6790, 2078, 4040, 4983, 6126, 694...",6,None,"[2078, 4017, 4018, 4033, 4036, 4040, 4055, 408...",None,511961571150868480,692104824862367744,None,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"[7538, 7563, 7602, 7812]","[167, 6820, 3242, 1992, 1352, 1110, 515, 4663,...",7,None,"[1067, 1110, 1352, 1386, 167, 1992, 2346, 3161...",None,694787445329293312,692104824862367744,None,0,...,0,0,0,on,on,NaN,NaN,NaN,NaN,NaN
7,"[7541, 7583, 7771, 7828]","[1166, 4034, 4988, 3199, 3321, 947, 3214, 1426...",8,None,"[1144, 1166, 1426, 2307, 2410, 3163, 3199, 320...",None,694977976969805824,692104824862367744,None,0,...,0,0,0,NaN,NaN,NaN,CMC,Reek,MT,D Hop
8,"[7572, 7581, 7603, 7694]","[96, 4199, 5000, 4131, 5859, 2374, 5012, 2449,...",9,[5284],"[138, 2197, 2374, 2449, 2749, 3423, 367, 4131,...",None,693183322456686592,692104824862367744,None,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,None,"[24, 4029, 3198, 5927, 1479, 3225, 1466, 2320,...",10,None,"[1049, 1466, 1479, 1848, 2320, 24, 2431, 3198,...",None,695046489558286336,692104824862367744,None,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
sleeper_players=requests.get(f"https://api.sleeper.app/v1/players/nfl")

In [138]:
sleeper_players_df=pd.json_normalize(sleeper_players.json(),max_level=0)

In [149]:
player_df=sleeper_players_df.T[0].apply(pd.Series)

In [212]:
draft_df['player_name']=draft_df['player_name'].replace(name_dict)

### remove the pick and add the player to the teams roster

In [162]:
u1=draft_df.iloc[2].players

In [169]:
u1_df=player_df.loc[u1]
u1_df['full_name'].values

array(['Amari Cooper', 'Dak Prescott', 'Leonard Fournette',
       'Anthony Firkser', 'Blake Jarwin', 'Dare Ogunbowale',
       'Christian Kirk', 'Ito Smith', 'Keke Coutee', 'Dwayne Haskins',
       'Noah Fant', 'Tony Pollard', 'Miles Sanders', 'Scott Miller',
       'Tua Tagovailoa', 'K.J. Hamler', 'J.K. Dobbins', "Ke'Shawn Vaughn",
       'DeeJay Dallas', 'Cam Akers', 'Tyler Johnson', 'Marquez Callaway',
       'Nick Bowers', 'Anthony Schwartz', 'Amari Rodgers',
       'Elijah Mitchell', 'Terrace Marshall', 'Kadarius Toney'],
      dtype=object)

In [185]:
team=ALL[ALL['Full Name'].isin(u1_df['full_name'].values)]

In [177]:
me[me['Position']=='QB'].iloc[3:]['Lifetime Value']

Series([], Name: Lifetime Value, dtype: float64)

In [208]:
cols=['QB Starting','QB Backup','QB Depth','RB Starting','RB Backup','RB Depth','WR Starting','WR Backup','WR Depth','TE Starting','TE Backup','TE Depth','Player Value']

In [179]:
teams=pd.DataFrame(columns=cols)

In [186]:
team_dict= {'QB Starting':team[team['Position']=='QB'].iloc[:2]['Lifetime Value'].sum(),
            'QB Backup':team[team['Position']=='QB'].iloc[2]['Lifetime Value'].sum(),
            'QB Depth':team[team['Position']=='QB'].iloc[3:]['Lifetime Value'].sum(),
            'RB Starting':team[team['Position']=='RB'].iloc[:3]['Lifetime Value'].sum(),
            'RB Backup':team[team['Position']=='RB'].iloc[3:5]['Lifetime Value'].sum(),
            'RB Depth':team[team['Position']=='RB'].iloc[5:]['Lifetime Value'].sum(),
            'WR Starting':team[team['Position']=='WR'].iloc[:5]['Lifetime Value'].sum(),
            'WR Backup':team[team['Position']=='WR'].iloc[5:7]['Lifetime Value'].sum(),
            'WR Depth':team[team['Position']=='WR'].iloc[7:]['Lifetime Value'].sum(),
            'TE Starting':team[team['Position']=='TE'].iloc[:1]['Lifetime Value'].sum(),
            'TE Backup':team[team['Position']=='TE'].iloc[1:2]['Lifetime Value'].sum(),
            'TE Depth':team[team['Position']=='TE'].iloc[2:]['Lifetime Value'].sum()
    
}

In [187]:
team_dict

{'QB Starting': 514.97,
 'QB Backup': 15.23,
 'QB Depth': 0.0,
 'RB Starting': 622.97,
 'RB Backup': 135.88,
 'RB Depth': 99.81,
 'WR Starting': 358.3,
 'WR Backup': 92.50999999999999,
 'WR Depth': 102.51,
 'TE Starting': 99.82,
 'TE Backup': 42.22,
 'TE Depth': 35.53}

In [191]:
teams=teams.append(pd.Series(team_dict,name=1))

In [192]:
teams

,QB Starting,QB Backup,QB Depth,RB Starting,RB Backup,RB Depth,WR Starting,WR Backup,WR Depth,TE Starting,TE Backup,TE Depth
1,514.97,15.23,0.0,622.97,135.88,99.81,358.3,92.51,102.51,99.82,42.22,35.53


### Lets automate it

In [217]:
teams=pd.DataFrame(columns=cols)

for x in range(12):
    u1=draft_df.iloc[x].players
    u1_df=player_df.loc[u1]
    team=ALL[ALL['Full Name'].isin(u1_df['full_name'].values)]
    team_dict= {'QB Starting':team[team['Position']=='QB'].iloc[:2]['Lifetime Value'].sum(),
            'QB Backup':team[team['Position']=='QB'].iloc[2]['Lifetime Value'].sum(),
            'QB Depth':team[team['Position']=='QB'].iloc[3:]['Lifetime Value'].sum(),
            'RB Starting':team[team['Position']=='RB'].iloc[:3]['Lifetime Value'].sum(),
            'RB Backup':team[team['Position']=='RB'].iloc[3:5]['Lifetime Value'].sum(),
            'RB Depth':team[team['Position']=='RB'].iloc[5:]['Lifetime Value'].sum(),
            'WR Starting':team[team['Position']=='WR'].iloc[:5]['Lifetime Value'].sum(),
            'WR Backup':team[team['Position']=='WR'].iloc[5:7]['Lifetime Value'].sum(),
            'WR Depth':team[team['Position']=='WR'].iloc[7:]['Lifetime Value'].sum(),
            'TE Starting':team[team['Position']=='TE'].iloc[:1]['Lifetime Value'].sum(),
            'TE Backup':team[team['Position']=='TE'].iloc[1:2]['Lifetime Value'].sum(),
            'TE Depth':team[team['Position']=='TE'].iloc[2:]['Lifetime Value'].sum(),
            'Player Value':team['Lifetime Value'].sum(),
            'Future Pick Value': sum([pick_val[y] for y in picks[x+1]])
                
            }
    teams=teams.append(pd.Series(team_dict,name=x))

In [218]:
teams.style.background_gradient(cmap='RdYlGn', high=.2,low=0.4)

,QB Starting,QB Backup,QB Depth,RB Starting,RB Backup,RB Depth,WR Starting,WR Backup,WR Depth,TE Starting,TE Backup,TE Depth,Player Value,Future Pick Value
0,348.160000,67.450000,65.050000,201.620000,47.970000,15.740000,559.590000,80.900000,73.970000,126.750000,18.380000,0.000000,1605.580000,288.000000
1,507.820000,21.180000,11.250000,360.590000,41.080000,24.310000,377.610000,45.090000,37.780000,143.270000,40.000000,31.190000,1641.170000,192.000000
2,514.970000,15.230000,0.000000,622.970000,135.880000,99.810000,358.300000,92.510000,102.510000,99.820000,42.220000,35.530000,2119.750000,339.000000
3,536.410000,49.250000,29.840000,307.530000,58.610000,29.060000,325.210000,63.980000,86.540000,105.610000,29.940000,28.790000,1650.770000,147.000000
4,456.430000,19.560000,28.730000,235.410000,38.740000,31.240000,455.370000,38.350000,0.000000,48.430000,46.510000,13.370000,1412.140000,96.000000
5,422.900000,150.860000,147.040000,590.830000,46.610000,34.020000,395.280000,86.220000,62.470000,58.900000,52.030000,95.010000,2142.170000,141.000000
6,240.240000,79.400000,75.230000,332.560000,93.080000,95.440000,453.470000,37.230000,29.840000,83.170000,51.680000,30.730000,1602.070000,384.000000
7,227.780000,16.130000,10.000000,510.810000,60.170000,37.010000,517.630000,54.410000,53.480000,58.090000,40.860000,31.290000,1617.660000,45.000000
8,444.590000,44.480000,30.610000,268.460000,39.550000,14.950000,606.470000,106.220000,112.130000,99.750000,19.940000,27.970000,1815.120000,192.000000
9,304.850000,38.560000,11.800000,402.480000,62.910000,52.280000,414.730000,38.930000,75.320000,156.650000,31.010000,27.020000,1616.540000,192.000000


Here We can see all the teams on the the different rows, and where their relative strengths and weakness are in the different positions to quickly find potential trade partners. 